<a href="https://colab.research.google.com/github/chanind/lc0_elo_testing/blob/main/Lc0_Elo_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leela Chess Zero Elo Testing

This Colab implements the ideas from the [Lc0 Testing Guide](https://lczero.org/dev/wiki/testing-guide/) to test out Lc0 by having it compete against other chess engines. Edit this Colab to test out different configurations or networks for Lc0, or use it as as base for your own explorations. The implementation presented here just runs standard Lc0 against standard Stockfish.


*Make sure to run this on a GPU instance to get the best performance from Lc0.*

In [ ]:
# Ensure that a GPU is available
import subprocess
smi=subprocess.run(["nvidia-smi", "-L"], stdout=subprocess.PIPE)
if smi.returncode != 0:
  assert False, "No GPU available!"
print(smi.stdout.decode())

In [1]:
# Install cutechess-cli, ordo, lc0, and stockfish
# This may take a few minute to run

# install Ordo
%cd /content
!rm -rf Ordo
!git clone --recurse-submodules https://github.com/michiguel/Ordo.git
%cd Ordo
! make
! make install

# install qt 5.12 for cutechess
! add-apt-repository -y ppa:beineri/opt-qt-5.12.6-bionic
! apt install qt512-meta-full

# install cutechess-cli
%cd /content
!rm -rf cutechess
!git clone --recurse-submodules https://github.com/cutechess/cutechess.git
%cd cutechess/projects
! /opt/qt512/bin/qmake -after "SUBDIRS = lib cli"
! make

# install lc0 v0.28
! apt install ninja-build
! pip3 install meson
%cd /content
!rm -rf lc0
! git clone -b release/0.28 --recurse-submodules https://github.com/LeelaChessZero/lc0.git
%cd lc0
! ./build.sh

# install stockfish 14.1
%cd /content
!wget https://stockfishchess.org/files/stockfish_14.1_linux_x64.zip
!unzip stockfish_14.1_linux_x64.zip
!cp stockfish_14.1_linux_x64/stockfish_14.1_linux_x64 stockfish
!chmod +x stockfish


/content
Cloning into 'Ordo'...
remote: Enumerating objects: 5889, done.
remote: Total 5889 (delta 0), reused 0 (delta 0), pack-reused 5889
Receiving objects: 100% (5889/5889), 1.68 MiB | 9.58 MiB/s, done.
Resolving deltas: 100% (2677/2677), done.
/content/Ordo
gcc -c -o myopt/myopt.o myopt/myopt.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport
gcc -c -o sysport/sysport.o sysport/sysport.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport
gcc -c -o mystr.o mystr.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport
gcc -c -o proginfo.o proginfo.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport
gcc -c -o pgnget.o pgnget.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport
gcc -c -o randfast.o randfast.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport
gcc -c -o gauss.o gauss.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport
gcc -c -o groups.o groups.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport
gcc -c -o cegt.o cegt.c -DNDEBUG -DMY_SEMAPHORES -flto -I myopt -I sysport

In [2]:
# download openings book
%cd /content
!wget https://cdn.discordapp.com/attachments/429710776282906625/536596158018224139/openings.zip
!unzip openings.zip

# download a net for lc0
# check https://lczero.org/play/networks/bestnets/ for more nets
%cd /content
! wget -O lc0-net http://training.lczero.org/get_network?sha=98412989c314a93aa4f2301faa0c8a93a95739ed10661f3674d7e923f269acd6

/content
--2022-02-20 22:03:56--  http://training.lczero.org/get_network?sha=98412989c314a93aa4f2301faa0c8a93a95739ed10661f3674d7e923f269acd6
Resolving training.lczero.org (training.lczero.org)... 188.114.97.0, 188.114.96.0, 2a06:98c1:3121::, ...
Connecting to training.lczero.org (training.lczero.org)|188.114.97.0|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /cached/network/sha/98412989c314a93aa4f2301faa0c8a93a95739ed10661f3674d7e923f269acd6 [following]
--2022-02-20 22:03:56--  http://training.lczero.org/cached/network/sha/98412989c314a93aa4f2301faa0c8a93a95739ed10661f3674d7e923f269acd6
Reusing existing connection to training.lczero.org:80.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://storage.lczero.org/files/networks/98412989c314a93aa4f2301faa0c8a93a95739ed10661f3674d7e923f269acd6 [following]
--2022-02-20 22:03:56--  http://storage.lczero.org/files/networks/98412989c314a93aa4f2301faa0c8a93a95739ed10661f3674

In [5]:
# Setup CuteChess engines.json config
# Edit this to change which engines are available for testing
CUTECHESS_ENGINES_JSON = """
[
    {
        "command": "/content/lc0/build/release/lc0",
        "name": "lc0",
        "options": [
            {
                "name": "WeightsFile",
                "value": "/content/lc0-net"
            },
            {
                "name": "Threads",
                "value": 2
            }
        ],
        "protocol": "uci",
        "workingDirectory": "/content"
    },
    {
        "command": "/content/stockfish",
        "name": "sf",
        "options": [
            {
                "name": "Hash",
                "value": 256
            },
            {
                "name": "Threads",
                "value": 12
            }
        ],
        "protocol": "uci",
        "workingDirectory": "/content"
    }
]
"""
with open('/content/engines.json', 'w') as config_file:
  config_file.write(CUTECHESS_ENGINES_JSON)

In [11]:
# Run the tournament!
# change the params below to adjust the rules of the tournament
# incrase "rounds" for more accurate but slower results
%cd /content
!/content/cutechess/projects/cli/cutechess-cli \
  -tournament gauntlet \
  -concurrency 1 \
  -pgnout output_pgn_file.pgn \
  -engine conf=sf tc=5.2+0.0832 \
  -engine conf=lc0 tc=4+0.064 \
  -draw movenumber=40 movecount=4 score=8 \
  -resign movecount=4 score=500 \
  -each proto=uci \
  -openings file=openings-6ply-1000.pgn \
  policy=round -repeat \
  -rounds 100 \
  -games 2

/content
Started game 1 of 200 (sf vs lc0)
Finished game 1 (sf vs lc0): 0-1 {Black wins by adjudication}
Score of sf vs lc0: 0 - 1 - 0  [0.000] 1
Started game 2 of 200 (lc0 vs sf)
Finished game 2 (lc0 vs sf): 0-1 {Black wins by adjudication}
Score of sf vs lc0: 1 - 1 - 0  [0.500] 2
Started game 3 of 200 (sf vs lc0)
Finished game 3 (sf vs lc0): 1-0 {White wins by adjudication}
Score of sf vs lc0: 2 - 1 - 0  [0.667] 3
Started game 4 of 200 (lc0 vs sf)
Finished game 4 (lc0 vs sf): 0-1 {Black wins by adjudication}
Score of sf vs lc0: 3 - 1 - 0  [0.750] 4
Started game 5 of 200 (sf vs lc0)
Finished game 5 (sf vs lc0): 1-0 {White wins by adjudication}
Score of sf vs lc0: 4 - 1 - 0  [0.800] 5
Started game 6 of 200 (lc0 vs sf)
Finished game 6 (lc0 vs sf): 0-1 {Black wins by adjudication}
Score of sf vs lc0: 5 - 1 - 0  [0.833] 6
Started game 7 of 200 (sf vs lc0)
Finished game 7 (sf vs lc0): 1-0 {White wins by adjudication}
Score of sf vs lc0: 6 - 1 - 0  [0.857] 7
Started game 8 of 200 (lc0 vs sf

In [13]:
# calculate Elo relative to stockfish
%cd /content
! ordo -Q -D -a 0 -A "sf" -W -n8 -s1000 -U "0,1,2,3,4,5,6,7,8,9,10" -j h2h.txt -C matrix.csv -p output_pgn_file.pgn

/content
0   10   20   30   40   50   60   70   80   90   100 (%)
|----|----|----|----|----|----|----|----|----|----|
***************************************************

   # PLAYER    :  RATING  ERROR  POINTS  PLAYED   (%)  CFS(%)    W    D    L  D(%)
   1 sf        :     0.0   ----   168.5     203    83     100  144   49   10    24
   2 lc0       :  -280.1   50.7    34.5     203    17     ---   10   49  144    24

White advantage = 34.94 +/- 24.63
Draw rate (equal opponents) = 39.59 % +/- 5.59

